<a href="https://colab.research.google.com/github/AmiraliLalehpour/MMAI-823-Final-Project/blob/main/Finance_Project_Price_Momentum_Strategy_Gurashish_Juneja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

import os

import plotly.express as px

import string
import spacy
from collections import defaultdict

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import re
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
current_directory = os.getcwd()
print(current_directory)

/content


In [25]:
path = "/content/drive/MyDrive/Finance Project/output.csv"

In [26]:
data = pd.read_csv(path)

In [27]:
data.head()

,title,score,id,url,comms_num,created,timestamp,combined_text,title_sentiment,combined_text_sentiment,title_overall_sentiment,combined_text_overall_sentiment
0,money sending message 🚀💎🙌,-0.234762,l6ulcx,https://v.redd.it/6j75regs72e61,-0.094876,1611862661,2021-01-28 21:37:00,money sending message 🚀💎🙌,0.0000,0.0000,Neutral,Neutral
1,math professor scott steiner say number spell ...,-0.211118,l6uibd,https://v.redd.it/ah50lyny62e61,-0.086817,1611862330,2021-01-28 21:32:00,math professor scott steiner say number spell ...,-0.5859,-0.5859,Negative,Negative
2,exit system,-0.258406,l6uhhn,https://www.reddit.com/r/wallstreetbets/commen...,-0.075440,1611862235,2021-01-28 21:30:00,exit system ceo nasdaq pushed halt trading “ g...,0.0000,0.4215,Neutral,Positive
3,new sec filing gme someone le retarded please ...,-0.245939,l6ugk6,https://sec.report/Document/0001193125-21-019848/,-0.062641,1611862137,2021-01-28 21:28:00,new sec filing gme someone le retarded please ...,-0.3400,-0.3400,Negative,Negative
4,distract gme thought amc brother aware,-0.227884,l6ufgy,https://i.redd.it/4h2sukb662e61.jpg,-0.023769,1611862016,2021-01-28 21:26:00,distract gme thought amc brother aware,-0.2960,-0.2960,Negative,Negative


In [28]:
data.shape

(52430, 12)

In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52430 entries, 0 to 52429
Data columns (total 12 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   title                            52326 non-null  object 
 1   score                            52430 non-null  float64
 2   id                               52430 non-null  object 
 3   url                              52430 non-null  object 
 4   comms_num                        52430 non-null  float64
 5   created                          52430 non-null  int64  
 6   timestamp                        52415 non-null  object 
 7   combined_text                    52368 non-null  object 
 8   title_sentiment                  52430 non-null  float64
 9   combined_text_sentiment          52430 non-null  float64
 10  title_overall_sentiment          52430 non-null  object 
 11  combined_text_overall_sentiment  52430 non-null  object 
dtypes: float64(4), int

Sentiment Signals

In [30]:
#We'll convert the sentiment scores (title_sentiment and combined_text_sentiment) into sentiment signals (+1 for positive, -1 for negative, 0 for neutral).

In [31]:
def get_sentiment_signal(score):
    if score > 0.05:   # Positive threshold (you can adjust this based on your sentiment analysis)
        return 1       # Positive sentiment signal
    elif score < -0.05:  # Negative threshold (you can adjust this based on your sentiment analysis)
        return -1      # Negative sentiment signal
    else:
        return 0       # Neutral sentiment signal

# Apply the function to create sentiment signals for both 'title' and 'combined_text'
data['title_sentiment_signal'] = data['title_sentiment'].apply(get_sentiment_signal)
data['combined_text_sentiment_signal'] = data['combined_text_sentiment'].apply(get_sentiment_signal)

In [32]:
data.head()

,title,score,id,url,comms_num,created,timestamp,combined_text,title_sentiment,combined_text_sentiment,title_overall_sentiment,combined_text_overall_sentiment,title_sentiment_signal,combined_text_sentiment_signal
0,money sending message 🚀💎🙌,-0.234762,l6ulcx,https://v.redd.it/6j75regs72e61,-0.094876,1611862661,2021-01-28 21:37:00,money sending message 🚀💎🙌,0.0000,0.0000,Neutral,Neutral,0,0
1,math professor scott steiner say number spell ...,-0.211118,l6uibd,https://v.redd.it/ah50lyny62e61,-0.086817,1611862330,2021-01-28 21:32:00,math professor scott steiner say number spell ...,-0.5859,-0.5859,Negative,Negative,-1,-1
2,exit system,-0.258406,l6uhhn,https://www.reddit.com/r/wallstreetbets/commen...,-0.075440,1611862235,2021-01-28 21:30:00,exit system ceo nasdaq pushed halt trading “ g...,0.0000,0.4215,Neutral,Positive,0,1
3,new sec filing gme someone le retarded please ...,-0.245939,l6ugk6,https://sec.report/Document/0001193125-21-019848/,-0.062641,1611862137,2021-01-28 21:28:00,new sec filing gme someone le retarded please ...,-0.3400,-0.3400,Negative,Negative,-1,-1
4,distract gme thought amc brother aware,-0.227884,l6ufgy,https://i.redd.it/4h2sukb662e61.jpg,-0.023769,1611862016,2021-01-28 21:26:00,distract gme thought amc brother aware,-0.2960,-0.2960,Negative,Negative,-1,-1


Sentiment Weighted Returns

In [33]:
data['title_sentiment_signal'] = data['title_sentiment'].apply(get_sentiment_signal)
data['combined_text_sentiment_signal'] = data['combined_text_sentiment'].apply(get_sentiment_signal)

In [34]:
# Calculate key metrics for sentiment-enhanced Price Momentum Strategy
title_sentiment_cumulative_return = data['title_cumulative_returns'].iloc[-1]
combined_text_sentiment_cumulative_return = data['combined_text_cumulative_returns'].iloc[-1]


KeyError: ignored

In [ ]:
# Calculate daily average compound score and generate sentiment signals
data_daily_compound = data_sorted_sentiment.groupby(pd.Grouper(key='timestamp', freq='D')).agg({'compound_score': 'mean'}).reset_index()
data_daily_compound['sentiment_signal'] = data_daily_compound['compound_score'].apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
data_daily_compound.set_index('timestamp', inplace=True)



